In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2023-07-18T00:18:24.040288-04:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.10.0

Compiler    : GCC 12.3.0
OS          : Linux
Release     : 5.15.0-76-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 16
Architecture: 64bit



In [3]:
%watermark --gpu

GPU Info: 
  GPU 0: Quadro RTX 5000



In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import sklearn
import gc
import os
import xgboost

In [5]:
%watermark --iversions

xgboost: 1.7.6
sklearn: 1.3.0
pandas : 2.0.3
numpy  : 1.25.1



In [6]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

In [7]:
train = pd.read_csv('../input/train.csv.zip')
test = pd.read_csv('../input/test.csv.zip')
sample_submission = pd.read_csv('../input/sample_submission.csv.zip')

In [8]:
columns = test.columns[1:]

In [9]:
X = train[columns].values
X_test = test[columns].values
Y = train.target.values

In [10]:
XGBClassifier()

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [11]:
%%time
train_oof = np.zeros((X.shape[0], ))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = XGBClassifier(n_estimators=750, max_depth=3, random_state=42, subsample=0.9, colsample_bytree=0.9, 
                          learning_rate=0.05)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)[:,1]
    train_oof[val_index] = val_pred
    print("Fold normalized:", gini_normalized(val_target, val_pred))
    test_preds += model.predict_proba(X_test)[:,1]/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

Fitting fold 1
Fold normalized: 0.2839817644792357
Fitting fold 2
Fold normalized: 0.288922224254601
Fitting fold 3
Fold normalized: 0.27280926254668875
Fitting fold 4
Fold normalized: 0.28080023317769315
Fitting fold 5
Fold normalized: 0.2775315118120522
CPU times: user 1h 13min 51s, sys: 2.83 s, total: 1h 13min 54s
Wall time: 4min 39s


In [12]:
gini_normalized(Y, train_oof)

0.28066020805306147

In [13]:
roc_auc_score(Y, train_oof)

0.6403301058349362

In [14]:
sample_submission['target'] = test_preds

In [15]:
sample_submission.to_csv('../submissions/XGB_5_fold.csv.zip', index=False, compression='zip')

On the leaderbaord this submission scores 0.28039 on the public test set, and 0.28435 on the private dataset.